Начальный импорт

In [2226]:
#Импортируем нужные библиотеки
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2227]:
#Оставляем только объявления в Москве и убираем, на наш взгляд, не нужные колонки.
df = pd.read_csv('./_data.csv', sep=',')
includes = []
for i in range(len(df)):
    if 'Москва' not in df['Адрес'][i]:
        includes.append(i)
print(df.columns.to_list())
df_clean=df.drop(includes)
print(len(df_clean))
df = df_clean.copy().drop(['Тип', 'Unnamed: 0', 'Телефоны', 'Серия дома', 'Название ЖК', 'Ссылка на объявление','Описание','Адрес','Дополнительно'], axis=1)
df['Ремонт'].fillna('неизвестно')

df.to_csv('data.csv', sep='\t')

['Unnamed: 0', 'ID  объявления', 'Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом', 'Парковка', 'Цена', 'Телефоны', 'Описание', 'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел', 'Можно с детьми/животными', 'Дополнительно', 'Название ЖК', 'Серия дома', 'Высота потолков, м', 'Лифт', 'Мусоропровод', 'Ссылка на объявление']
19737


Работа с NaN

In [2228]:
#Убираем дубликаты
df = df.drop_duplicates()

In [2229]:
#Убираем NaN в кол-ве комнат и приводим кол-во комнат в числовое или около того значение.
df['Количество комнат'] = df['Количество комнат'].str[0]
df['Количество комнат'].fillna(int(df['Количество комнат'].mode()), inplace=True)

In [2230]:
# Считаем, что отдельные площади кухни и комнат нам не важны, и оставляет только общую площадь, а потом удаляем колонки со старой площадью.
df['Площадь кв, м2'] = [row.split('/')[0] for row in df['Площадь, м2']]
df = df.drop(['Площадь, м2'], axis = 1)
df = df.drop(['Площадь комнат, м2'], axis = 1)

In [2231]:
# Убираем NaN в колонке 'Окна', ставим 'Во двор', так как это самый распространённый вариант.
df['Окна'] = df['Окна'].fillna('Во двор')

In [2232]:
# Убираем NaN в колонке 'Ремонт', ставим 'Косметический', так как это самый распространённый вариант.
df['Ремонт'] = df['Ремонт'].fillna('Косметический')

In [2233]:
# Вытаскиваем информацию из колонки 'дом'
df['Тип дома'] = [row.split(', ')[-1] if len(row.split(', ')[-1]) > 6 else 'Неизвестно' for row in df['Дом']]
df['Этажность дома'] = [int(row.split('/')[1].split(',')[0]) for row in df['Дом']]
df['Этаж квартиры'] = [int(row.split('/')[0]) for row in df['Дом']]
df = df.drop(['Дом'], axis = 1)

In [2234]:
# Избавляемся от NaN для лифтов, если меньше 5 этажей - нет лифта, в остальном по одному каждого лифта.
df['Лифт'] = [row if row == row else
              ['Пасс (1), Груз (1)' if df['Этажность дома'].iloc[i]>5 else 'Пасс (0), Груз (0)'][0]
              for i, row in enumerate(df['Лифт'])]

In [2235]:
# Убираем NaN в колонке 'Мусоропровод', ставим 'Нет', так как предполагаем, что это малозначимый фактор.
df['Мусоропровод'] = df['Мусоропровод'].fillna('Нет')

In [2236]:
# Убираем NaN в колонке 'Высота потолков, м', выбираем среднее, так как в случае с высотой потолков это кажется самым логичным вариантом.
df['Высота потолков, м'] = df['Высота потолков, м'].fillna(round(df['Высота потолков, м'].mean(), 2))


In [2237]:
# Убираем NaN в колонке 'Санузел', разбиваем на возможные варианты, руководствуясь логикой.
df['Санузел'] = df['Санузел'].fillna('Совмещенный (1)')
df['Санузел совмещенный'] = [row.split(' ')[1] if (row.split(' ')[0]) == 'Совмещенный' else '(0)' for row in df['Санузел']]
df['Санузел раздельный'] = [row.split(' ')[-1] if (row.split(' ')[-2]) == 'Раздельный' else '(0)' for row in df['Санузел']]
df['Санузел совмещенный'] = [row.split('(')[1] for row in df['Санузел совмещенный']]
df['Санузел совмещенный'] = [int(row.split(')')[0]) for row in df['Санузел совмещенный']]
df['Санузел раздельный'] = [row.split('(')[1] for row in df['Санузел раздельный']]
df['Санузел раздельный'] = [int(row.split(')')[0]) for row in df['Санузел раздельный']]
df = df.drop(['Санузел'], axis = 1)

In [2238]:
# Убираем NaN в колонке 'Балкон', разбиваем на возможные варианты, руководствуясь логикой.
df['Балкон'] = df['Балкон'].fillna('Балкон (0), Лоджия (0)')
df['Лоджия'] = [row.split(' ')[-1] if (row.split(' ')[-2]) == 'Лоджия' else '(0)' for row in df['Балкон']]
df['Балкон'] = [row.split(' ')[1] if row.split(' ')[0] == 'Балкон' else '(0)' for row in df['Балкон']]
df['Лоджия'] = [row.split('(')[1] for row in df['Лоджия']]
df['Лоджия'] = [int(row.split(')')[0]) for row in df['Лоджия']]
df['Балкон'] = [row.split('(')[1] for row in df['Балкон']]
df['Балкон'] = [int(row.split(')')[0]) for row in df['Балкон']]

In [2239]:
# Убираем NaN в колонке 'Парковка', ставим 'Наземная', так как это самый распространённый вариант и он бесплатный.
df['Парковка'] = df['Парковка'].fillna('наземная')
df['Парковка'] = df['Парковка'].apply(lambda x: x.capitalize())

In [2240]:
# Убираем NaN в колонке 'Можно с детьми/животными', в данном случае мы считаем фактор важным, поэтому никак его автоматически не заполняем.
df['Можно с детьми/животными']=df['Можно с детьми/животными'].fillna('по договорённости')

In [2241]:
#Переводим расстояние до метро в метры в зависимости от указанного способо передвижения, для NaN задаём медиану.
df['время до метро'] = df['Метро'].str.replace(')', '').str.split('(').str[-1]
df[['время в минутах', 'способ']] = df['время до метро'].str.extract(r'(\d+) мин\s(пешком|на машине)')
df['способ'] = df['способ'].apply(lambda x: 70 if x == 'пешком' else 750)
df['способ'] = df['способ'].fillna(0)
df['время в минутах'] = df['время в минутах'].fillna(0)
df['distance to subway, m'] = df['способ'] * df['время в минутах'].astype(int)
median_value = df['distance to subway, m'].median()
df['distance to subway, m'] = df['distance to subway, m'].replace(0, median_value)


/tmp/ipykernel_37024/2430123984.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['время до метро'] = df['Метро'].str.replace(')', '').str.split('(').str[-1]


In [2242]:
#Переименовываем столбцы на английский для соответствия требованиям.
df.rename(columns={'ID  объявления': 'Ad_ID'}, inplace=True)
df.rename(columns={'Количество комнат': 'Number_of_Rooms'}, inplace=True)
df.rename(columns={'Метро': 'Metro'}, inplace=True)
df.rename(columns={'Парковка': 'Parking'}, inplace=True)
df.rename(columns={'Цена': 'Price'}, inplace=True)
df.rename(columns={'Ремонт': 'Renovation'}, inplace=True)
df.rename(columns={'Балкон': 'Balcony'}, inplace=True)
df.rename(columns={'Окна': 'Windows'}, inplace=True)
df.rename(columns={'Можно с детьми/животными': 'Children/Pets_Allowed'}, inplace=True)
df.rename(columns={'Лифт': 'Elevator'}, inplace=True)
df.rename(columns={'Мусоропровод': 'Garbage_Chute'}, inplace=True)
df.rename(columns={'Площадь кв, м2': 'Area_sq_m'}, inplace=True)
df.rename(columns={'Тип дома': 'House_Type'}, inplace=True)
df.rename(columns={'Этажность дома': 'Number_of_Floors'}, inplace=True)
df.rename(columns={'Этаж квартиры': 'Floor_of_Apartment'}, inplace=True)
df.rename(columns={'Санузел совмещенный': 'Combined_Bathroom'}, inplace=True)
df.rename(columns={'Санузел раздельный': 'Separate_Bathroom'}, inplace=True)
df.rename(columns={'Лоджия': 'Balconette'}, inplace=True)
df.rename(columns={'Высота потолков, м': 'Ceiling_Height_M'}, inplace=True)


In [2243]:
#Убираем лишние столбцы.
df.drop(['время до метро'], axis=1, inplace = True)
df.drop(['время в минутах'], axis=1, inplace = True)
df.drop(['способ'], axis=1, inplace = True)
df.drop(['Metro'], axis=1, inplace = True)

In [2244]:
#Проверка на NaN
nans = df.isna().any().any()
print(nans)


False


In [2245]:
#Разбивка лифтов на пассажирские и грузовые
df.insert(loc=df.columns.to_list().index('Elevator')+1, column='Count_freight_elevator', \
                  value=df['Elevator'].apply(lambda x: int(x[-2]) if 'Груз' in x else 0))
df['Elevator'] = df['Elevator'].apply(lambda x: int(x[x.index('(')+1]) if 'Пасс' in x else 0)
df = df.rename(columns={'Elevator': 'Count_pass_elevator'})

Работа с кодированием

In [2246]:
#Создаём копию основного датасета, для того что бы его не изменять.
df_encode = df.copy()

In [2247]:
# Кодирование Parking
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['Parking'])
df_encode['Parking'] = le.transform(df_encode['Parking'])



In [2248]:
# Кодирование Renovation
le = LabelEncoder()
le.fit(df['Renovation'])
df_encode['Renovation'] = le.transform(df_encode['Renovation'])

In [2249]:
# Кодирование Garbage_Chute
le = LabelEncoder()
le.fit(df['Garbage_Chute'])
df_encode['Garbage_Chute'] = le.transform(df_encode['Garbage_Chute'])

In [2250]:
# кодирование House_Type
le = LabelEncoder()
le.fit(df['House_Type'])
df_encode['House_Type'] = le.transform(df_encode['House_Type'])

In [2251]:
# кодирование Windows
le = LabelEncoder()
le.fit(df['Windows'])
df_encode['Windows'] = le.transform(df_encode['Windows'])

In [2252]:
#Переводим столбец Number_of_Rooms и Area_sq_m из str в int
df_encode['Number_of_Rooms'] = df_encode['Number_of_Rooms'].astype(int)
df_encode['Area_sq_m'] = df_encode['Area_sq_m'].astype(float)

In [2253]:
# Разбиение одной колонки на две
df_encode['Childrens_allowed'] = df_encode['Children/Pets_Allowed'].str.contains('Можно с детьми', case=False)
df_encode['Pets_allowed'] = df_encode['Children/Pets_Allowed'].str.contains('Можно с животными', case=False)

In [2254]:
# выбрасываем то, что разбивали
df_encode.drop('Children/Pets_Allowed', axis=1, inplace=True)

In [2255]:
#str to int
df_encode['Childrens_allowed'] = df_encode['Childrens_allowed'].astype(int)
df_encode['Pets_allowed'] = df_encode['Pets_allowed'].astype(int)

In [2256]:
# Добавим новый столбец Deposit
df.insert(loc=df.columns.to_list().index('Price')+1, column="Deposit",
         value=df['Price'])

# Переведём столбец Price из $ и € в рубли (где необходимо)
def converter(x):
    x = x.split(' ')
    
    if x[1] == '$':
        return int(float((x[0]))) * 95
        
    elif x == '€':
        return int(float((x[0]))) * 100
        
    else:
        return int(float((x[0])))

df['Price'] = df['Price'].apply(converter)

In [2257]:
# Отредактируем cтолбец Deposit

from string import punctuation

# Удалим ненужную пунктуацию, буквы и отступы на число
def del_punc(x):
    for let in x[:]:
        if let in punctuation:
            x = x.replace(let, '')
            
    return x

def cleaner_deposit(x):
    if 'Залог' in x:
        x = x.split('Залог')
        if '$' in x[1]:
            x = del_punc(x[1]).split()
            return int(x[0]) * 95
        
        elif '€' in x[1]:
            x = del_punc(x[1]).split()
            return int(x[0]) * 100

        else:
            x = del_punc(x[1]).split()
            return int(x[0])
    else:
        return 0


df['Deposit'] = df['Deposit'].apply(cleaner_deposit)

In [2258]:
# Удаление несуществующих зданий по высоте (самое высокое жилое здание Мск - 95 этажей)
df_encode = df_encode[df_encode['Number_of_Floors'] <= 95]
len(df_encode)

19736

In [2259]:
# Удаление несуществующих квартир по площади (даже в общежитии 6м2 на человека. Ну зададимся 8 - самое минимальное, что видела Москва)
df_encode = df_encode[df_encode['Area_sq_m'] >= 8]
len(df_encode)

19735

In [2260]:
# Удаление квартир для гномов
df_encode = df_encode[df_encode['Ceiling_Height_M'] >= 2.2]
len(df_encode)

# Удаление квартир с 4-х метровыми колоннами 
df_encode = df_encode[df_encode['Ceiling_Height_M'] <= 8]
len(df_encode)

19714

In [2261]:
#Обновляем датасет
df.to_csv('data.csv', sep=',')
